In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
from pymongo import MongoClient
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
from tqdm import tqdm

In [2]:
client = MongoClient('localhost', 27017)
db = client.TFE
collection = db.test

In [3]:
def generateFromCM(cm):
    tn, fp, fn, tp = cm
    y_true = ['reliable' for i in range(0, tn + fp)]
    y_pred = ['reliable' for i in range(0, tn)]+['fake' for i in range(0, fp)]
    y_true = y_true + ['fake' for i in range(0, fn + tp)]
    y_pred = y_pred + ['reliable' for i in range(0, fn)] + ['fake' for i in range(0, tp)]
    return y_true, y_pred

In [4]:
cursor = collection.find({'model' : 'Self_Embedding LSTM 1.2', 'finish' : True}, no_cursor_timeout=True)
pbar = tqdm(total=1)
for i, res in enumerate(cursor):
    for i, result in enumerate(res['result']):
        epoch = result['epoch']
        cm = result['valid']['Confusion Matrix']
        y_true, y_pred = generateFromCM(cm)
        clr = classification_report(y_true, y_pred, output_dict = True)
        collection.update_one({'experiment_id' : res['experiment_id'], 'result.epoch' : epoch}, {"$set" : {'result.$.valid.clr' : clr}})
        
        cm = result['train']['Confusion Matrix']
        y_true, y_pred = generateFromCM(cm)
        clr = classification_report(y_true, y_pred, output_dict = True)
        collection.update_one({'experiment_id' : res['experiment_id'], 'result.epoch' : epoch}, {"$set" : {'result.$.train.clr' : clr}})
    pbar.update(1)
pbar.close()
cursor.close()

100%|██████████| 1/1 [00:34<00:00, 34.67s/it]


In [8]:
clr

{'fake': {'precision': 0.9239253483092063,
  'recall': 0.9339965220949263,
  'f1-score': 0.9289336389677527,
  'support': 156416},
 'reliable': {'precision': 0.9332691275992011,
  'recall': 0.923096102700491,
  'f1-score': 0.9281547406686016,
  'support': 156416},
 'accuracy': 0.9285463123977087,
 'macro avg': {'precision': 0.9285972379542037,
  'recall': 0.9285463123977087,
  'f1-score': 0.9285441898181772,
  'support': 312832},
 'weighted avg': {'precision': 0.9285972379542037,
  'recall': 0.9285463123977087,
  'f1-score': 0.9285441898181771,
  'support': 312832}}